# 尝试不同的MMD对齐方法

## 1.数据准备
保存到: /data3/fanpeishan/state/for_state/run_results/run15/c37_prep.h5ad
## 2.数据拆分
源域：/data3/fanpeishan/state/for_state/run_results/run20/data/source_domain.h5ad
目标域：/data3/fanpeishan/state/for_state/run_results/run20/data/target_align.h5ad
测试集：/data3/fanpeishan/state/for_state/run_results/run20/data/target_test.h5ad

## 3.1 train MMD adapter:affine

In [ ]:
# 使用 source_domain 和 target_align 数据训练 MMD 适配器
export CUDA_VISIBLE_DEVICES=3
python /data3/fanpeishan/state/for_state/MMD_alignment_experiment/scripts/train_mmd_adapter.py \
  --source_data /data3/fanpeishan/state/for_state/run_results/run20/data/source_domain.h5ad \
  --target_data /data3/fanpeishan/state/for_state/run_results/run20/data/target_align.h5ad \
  --output_dir /data3/fanpeishan/state/for_state/run_results/run22/adapters_affine/ \
  --pert_col drugname_drugconc \
  --control_name "[('DMSO_TF', 0.0, 'uM')]" \
  --adapter_type affine \
  --epochs 1000 \
  --lr 1e-4 \
  --log_interval 200 \
  --source_sample_size 15000 \
  --target_sample_size 10000 \
  --seed 42 

In [ ]:
============================================================
开始 MMD 适配器训练（精简整合版）
============================================================
加载数据: /data3/fanpeishan/state/for_state/run_results/run20/data/source_domain.h5ad
原始数据形状: (1193354, 2000)
控制组细胞数: 30136 / 1193354
随机采样到 15000 个细胞
特征形状: (15000, 2000)
加载数据: /data3/fanpeishan/state/for_state/run_results/run20/data/target_align.h5ad
原始数据形状: (321296, 2000)
控制组细胞数: 7503 / 321296
特征形状: (7503, 2000)
--------------------------------------------------
使用设备: cuda
训练配置:
  适配器类型: affine
  特征维度: 2000
  源域样本数: 15000
  目标域样本数: 7503
  训练轮数: 1000
  学习率: 0.0001
适配器参数数量: 4000
使用中位数准则估计MMD带宽...
带宽集合: ['7.6359', '15.2718', '30.5436', '61.0872', '122.1745']
初始MMD²: 0.000748
--------------------------------------------------
Epoch [   1/1000] MMD²: 0.000748 (改善: +0.00%) Best: 0.000748
Epoch [ 200/1000] MMD²: 0.000469 (改善: +37.23%) Best: 0.000469
Epoch [ 400/1000] MMD²: 0.000290 (改善: +61.22%) Best: 0.000290
Epoch [ 600/1000] MMD²: 0.000175 (改善: +76.58%) Best: 0.000175
Epoch [ 800/1000] MMD²: 0.000105 (改善: +85.90%) Best: 0.000105
Epoch [1000/1000] MMD²: 0.000066 (改善: +91.14%) Best: 0.000066
--------------------------------------------------
训练完成！初始MMD²: 0.000748 → 最终: 0.000066
MMD降低: 91.14%
适配器已保存至: /data3/fanpeishan/state/for_state/run_results/run22/adapters_affine/adapter_affine_20251223_231545_final.pt
训练成功完成！

## 4.1 apply MMD alignment:affine

In [ ]:
# 使用训练好的适配器,将目标域数据对齐到源域
export CUDA_VISIBLE_DEVICES=3
python /data3/fanpeishan/state/for_state/MMD_alignment_experiment/scripts/apply_mmd_alignment.py \
  --input_data /data3/fanpeishan/state/for_state/run_results/run20/data/target_test.h5ad \
  --adapter_path /data3/fanpeishan/state/for_state/run_results/run22/adapters_affine/adapter_affine_20251223_231545_final.pt \
  --output_data /data3/fanpeishan/state/for_state/run_results/run22/target_test_affine_aligned.h5ad \
  --adapter_type affine \
  --batch_size 10000 \
  --seed 42

In [ ]:
使用设备: cuda
============================================================
开始 MMD 对齐应用（精简整合版）
============================================================
输入数据: /data3/fanpeishan/state/for_state/run_results/run20/data/target_test.h5ad
适配器: /data3/fanpeishan/state/for_state/run_results/run22/adapters_affine/adapter_affine_20251223_231545_final.pt (类型: affine)
输出数据: /data3/fanpeishan/state/for_state/run_results/run22/target_test_affine_aligned.h5ad
------------------------------------------------------------
加载输入数据...
数据形状: (321297, 2000)
特征维度: 2000
特征范围: [0.000, 184.255]
------------------------------------------------------------
加载适配器...
适配器加载成功
------------------------------------------------------------
应用对齐转换...
批量对齐: 321297 个样本, 33 个批次 (batch_size=10000)
  处理批次 10/33
  处理批次 20/33
  处理批次 30/33
  处理批次 33/33
对齐转换完成
对齐后特征范围: [-0.107, 191.947]
------------------------------------------------------------
保存对齐结果...
警告: 原始 .X 将被覆盖！建议提前备份原始数据
数据已保存: /data3/fanpeishan/state/for_state/run_results/run22/target_test_affine_aligned.h5ad
============================================================
对齐成功完成！
============================================================

## 5.1 infer

In [ ]:
# 使用 MMD 对齐后的数据进行零样本推理
export CUDA_VISIBLE_DEVICES=3
state tx infer \
  --model-dir /data3/fanpeishan/state/for_state/models/ST-Tahoe \
  --checkpoint /data3/fanpeishan/state/for_state/models/ST-Tahoe/final_from_preprint.ckpt \
  --adata /data3/fanpeishan/state/for_state/run_results/run22/target_test_affine_aligned.h5ad \
  --output /data3/fanpeishan/state/for_state/run_results/run22/target_test_affine_aligned_infer.h5ad \
  --pert-col drugname_drugconc \
  --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
  --quiet \
  --batch-col plate

In [ ]:
=== Inference complete ===
Input cells:         321297
Controls simulated:  7511
Treated simulated:   313786
Wrote predictions to adata.X
Saved:               /data3/fanpeishan/state/for_state/run_results/run22/target_test_affine_aligned_infer.h5ad

## 6.1 eval

In [ ]:
# 使用 cell-eval 评估对齐组的推理结果
export CUDA_VISIBLE_DEVICES=3
cell-eval run \
    -ap /data3/fanpeishan/state/for_state/run_results/run22/target_test_affine_aligned_infer.h5ad \
    -ar /data3/fanpeishan/state/for_state/run_results/run20/data/target_test.h5ad \
    -o /data3/fanpeishan/state/for_state/run_results/run22/run_results_affine/ \
    --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
    --pert-col 'drugname_drugconc' \
    --profile minimal \
    --batch-size 1024 \
    --num-threads 12 \
    --skip-metrics de_nsig_counts_real,de_nsig_counts_pred

In [2]:
import pandas as pd
results = pd.read_csv('/data3/fanpeishan/state/for_state/run_results/run22/run_results_affine/agg_results.csv')
results[results.statistic == 'mean']

,statistic,overlap_at_N,precision_at_N,de_nsig_counts_real,de_nsig_counts_pred,pearson_delta,mse,mae,discrimination_score_l1
2,mean,0.122612,0.167219,192.318872,542.863341,0.350182,0.135218,0.084798,0.615266


## 3.2 train MMD adapter:mlp

In [ ]:
# 使用 source_domain 和 target_align 数据训练 MMD 适配器
export CUDA_VISIBLE_DEVICES=4
python /data3/fanpeishan/state/for_state/MMD_alignment_experiment/scripts/train_mmd_adapter.py \
  --source_data /data3/fanpeishan/state/for_state/run_results/run20/data/source_domain.h5ad \
  --target_data /data3/fanpeishan/state/for_state/run_results/run20/data/target_align.h5ad \
  --output_dir /data3/fanpeishan/state/for_state/run_results/run22/adapters_mlp/ \
  --pert_col drugname_drugconc \
  --control_name "[('DMSO_TF', 0.0, 'uM')]" \
  --adapter_type mlp \
  --epochs 1000 \
  --lr 1e-4 \
  --log_interval 200 \
  --source_sample_size 15000 \
  --target_sample_size 10000 \
  --seed 42 

In [ ]:
============================================================
开始 MMD 适配器训练（精简整合版）
============================================================
加载数据: /data3/fanpeishan/state/for_state/run_results/run20/data/source_domain.h5ad
原始数据形状: (1193354, 2000)
控制组细胞数: 30136 / 1193354
随机采样到 15000 个细胞
特征形状: (15000, 2000)
加载数据: /data3/fanpeishan/state/for_state/run_results/run20/data/target_align.h5ad
原始数据形状: (321296, 2000)
控制组细胞数: 7503 / 321296
特征形状: (7503, 2000)
--------------------------------------------------
使用设备: cuda
训练配置:
  适配器类型: mlp
  特征维度: 2000
  源域样本数: 15000
  目标域样本数: 7503
  训练轮数: 1000
  学习率: 0.0001
适配器参数数量: 4003000
使用中位数准则估计MMD带宽...
带宽集合: ['7.6359', '15.2718', '30.5436', '61.0872', '122.1745']
初始MMD²: 0.000748
--------------------------------------------------
Epoch [   1/1000] MMD²: 0.000748 (改善: +0.01%) Best: 0.000748
Epoch [ 200/1000] MMD²: -0.000006 (改善: +100.86%) Best: -0.000006
Epoch [ 400/1000] MMD²: -0.000015 (改善: +102.01%) Best: -0.000015
Epoch [ 600/1000] MMD²: -0.000022 (改善: +102.90%) Best: -0.000022
Epoch [ 800/1000] MMD²: -0.000027 (改善: +103.66%) Best: -0.000027
Epoch [1000/1000] MMD²: -0.000031 (改善: +104.09%) Best: -0.000031
--------------------------------------------------
训练完成！初始MMD²: 0.000748 → 最终: -0.000031
MMD降低: 104.11%
适配器已保存至: /data3/fanpeishan/state/for_state/run_results/run22/adapters_mlp/adapter_mlp_20251223_231617_final.pt
训练成功完成！

## 4.2 apply MMD alignment:mlp

In [ ]:
# 使用训练好的适配器,将目标域数据对齐到源域
export CUDA_VISIBLE_DEVICES=3
python /data3/fanpeishan/state/for_state/MMD_alignment_experiment/scripts/apply_mmd_alignment.py \
  --input_data /data3/fanpeishan/state/for_state/run_results/run20/data/target_test.h5ad \
  --adapter_path /data3/fanpeishan/state/for_state/run_results/run22/adapters_mlp/adapter_mlp_20251223_231617_final.pt \
  --output_data /data3/fanpeishan/state/for_state/run_results/run22/target_test_mlp_aligned.h5ad \
  --adapter_type mlp \
  --batch_size 10000 \
  --seed 42

In [ ]:
使用设备: cuda
============================================================
开始 MMD 对齐应用（精简整合版）
============================================================
输入数据: /data3/fanpeishan/state/for_state/run_results/run20/data/target_test.h5ad
适配器: /data3/fanpeishan/state/for_state/run_results/run22/adapters_mlp/adapter_mlp_20251223_231617_final.pt (类型: mlp)
输出数据: /data3/fanpeishan/state/for_state/run_results/run22/target_test_mlp_aligned.h5ad
------------------------------------------------------------
加载输入数据...
数据形状: (321297, 2000)
特征维度: 2000
特征范围: [0.000, 184.255]
------------------------------------------------------------
加载适配器...
适配器加载成功
------------------------------------------------------------
应用对齐转换...
批量对齐: 321297 个样本, 33 个批次 (batch_size=10000)
  处理批次 10/33
  处理批次 20/33
  处理批次 30/33
  处理批次 33/33
对齐转换完成
对齐后特征范围: [-11.388, 192.863]
------------------------------------------------------------
保存对齐结果...
警告: 原始 .X 将被覆盖！建议提前备份原始数据
数据已保存: /data3/fanpeishan/state/for_state/run_results/run22/target_test_mlp_aligned.h5ad
============================================================
对齐成功完成！
============================================================

## 5.2 infer

In [ ]:
# 使用 MMD 对齐后的数据进行零样本推理
export CUDA_VISIBLE_DEVICES=4
state tx infer \
  --model-dir /data3/fanpeishan/state/for_state/models/ST-Tahoe \
  --checkpoint /data3/fanpeishan/state/for_state/models/ST-Tahoe/final_from_preprint.ckpt \
  --adata /data3/fanpeishan/state/for_state/run_results/run22/target_test_mlp_aligned.h5ad \
  --output /data3/fanpeishan/state/for_state/run_results/run22/target_test_mlp_aligned_infer.h5ad \
  --pert-col drugname_drugconc \
  --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
  --quiet \
  --batch-col plate

In [ ]:
=== Inference complete ===
Input cells:         321297
Controls simulated:  7511
Treated simulated:   313786
Wrote predictions to adata.X
Saved:               /data3/fanpeishan/state/for_state/run_results/run22/target_test_mlp_aligned_infer.h5ad

## 6.2 eval

In [ ]:
# 使用 cell-eval 评估对齐组的推理结果
export CUDA_VISIBLE_DEVICES=4
cell-eval run \
    -ap /data3/fanpeishan/state/for_state/run_results/run22/target_test_mlp_aligned_infer.h5ad \
    -ar /data3/fanpeishan/state/for_state/run_results/run20/data/target_test.h5ad \
    -o /data3/fanpeishan/state/for_state/run_results/run22/run_results_mlp/ \
    --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
    --pert-col 'drugname_drugconc' \
    --profile minimal \
    --batch-size 1024 \
    --num-threads 12 \
    --skip-metrics de_nsig_counts_real,de_nsig_counts_pred

In [1]:
import pandas as pd
results = pd.read_csv('/data3/fanpeishan/state/for_state/run_results/run22/run_results_mlp/agg_results.csv')
results[results.statistic == 'mean']

,statistic,overlap_at_N,precision_at_N,de_nsig_counts_real,de_nsig_counts_pred,pearson_delta,mse,mae,discrimination_score_l1
2,mean,0.121803,0.166973,192.318872,540.236443,0.361899,0.131791,0.082787,0.616819
